In [1]:
import json
from typing import List

from tqdm.notebook import tqdm
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain.pydantic_v1 import BaseModel, Field
from langchain.schema import HumanMessage, AIMessage, StrOutputParser
import pandas as pd

In [2]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=1.0)

In [3]:
class Turn(BaseModel):
    role: str = Field("사람 이름")
    content: str = Field("대화 내용")
    
class Conversation(BaseModel):
    conversation: List[Turn] = Field("두 사람의 대화 내용")

In [4]:
parser = JsonOutputParser(pydantic_object=Conversation)

In [5]:
format_instruction = parser.get_format_instructions()

## Fewshot Example 만들기

In [6]:
prompt_template = PromptTemplate.from_template(template="영어 단어'{input}'에 대한 실생활 사용 예시 대화를 만들어줘. 아래 포맷으로 출력해줘.\n{format_instruction}",
                                               partial_variables={"format_instruction": format_instruction})

In [7]:
chain = {"input": RunnablePassthrough()}| prompt_template | model | parser

In [8]:
example_1 = chain.invoke("question")

In [9]:
example_1

{'conversation': [{'role': 'User', 'content': 'Can I ask you a question?'},
  {'role': 'Assistant', 'content': 'Of course! What do you need help with?'},
  {'role': 'User',
   'content': 'I am having trouble understanding this concept.'},
  {'role': 'Assistant',
   'content': 'Sure! What specific aspect are you having difficulty with?'}]}

In [10]:
example_2 = chain.invoke("occur")

In [11]:
example_2

{'conversation': [{'role': 'Person A',
   'content': 'Hey, did you see what happened last night?'},
  {'role': 'Person B', 'content': 'No, what occurred?'},
  {'role': 'Person A',
   'content': 'There was a car accident near the intersection.'},
  {'role': 'Person B', 'content': 'Oh no! Did anyone get hurt?'}]}

## Fewshot Example 적용해서 샘플링하기

In [12]:
example_1_json = json.dumps(example_1)
example_2_json = json.dumps(example_2)

In [13]:
example_1_json

'{"conversation": [{"role": "User", "content": "Can I ask you a question?"}, {"role": "Assistant", "content": "Of course! What do you need help with?"}, {"role": "User", "content": "I am having trouble understanding this concept."}, {"role": "Assistant", "content": "Sure! What specific aspect are you having difficulty with?"}]}'

In [14]:
example_2_json

'{"conversation": [{"role": "Person A", "content": "Hey, did you see what happened last night?"}, {"role": "Person B", "content": "No, what occurred?"}, {"role": "Person A", "content": "There was a car accident near the intersection."}, {"role": "Person B", "content": "Oh no! Did anyone get hurt?"}]}'

In [16]:
msgs = [
    ("human", "영어 단어'question'에 대한 실생활 사용 예시 대화를 만들어줘. 아래 포맷으로 출력해줘.\n{format_instruction}"),
    ("ai", "{example_1_json}"),
    ("human", "영어 단어'occur'에 대한 실생활 사용 예시 대화를 만들어줘. 아래 포맷으로 출력해줘.\n{format_instruction}"),
    ("ai", "{example_2_json}"),
    ("human", "영어 단어'{input}'에 대한 실생화 사용 예시 대화를 만들어줘. 아래 포맷으로 출력해줘.\n{format_instruction}")
]

In [17]:
chat_prompt_template = ChatPromptTemplate.from_messages(msgs)

In [29]:
fewshot_chain = {"input": RunnablePassthrough(),
                 "format_instruction": lambda _:format_instruction,
                 "example_1_json": lambda _:example_1_json, 
                 "example_2_json": lambda _:example_2_json}  | chat_prompt_template | model | parser

In [30]:
out = fewshot_chain.invoke("multiple")
out

{'conversation': [{'role': 'Person A',
   'content': 'I heard there will be multiple guest speakers at the conference.'},
  {'role': 'Person B',
   'content': "That's great! It will provide a diverse range of perspectives."},
  {'role': 'Person A',
   'content': 'Yes, it will be a valuable opportunity to learn from multiple experts in the field.'},
  {'role': 'Person B', 'content': "I'm looking forward to it!"}]}

## Load Excel and Save

In [20]:
df = pd.read_excel("./words.xlsx")

In [25]:
df

,word,imoj,meaning
0,effort,💪,effort은 노력이나 수고를 의미합니다. 그것은 어떤 일을 이루기 위해 투자한 에...
1,excitement,🎉,excitement은 흥분이나 설렘을 의미합니다. 어떤 일이나 사건이 특히 기쁘거나...
2,love,❤️,"love은 강한 애정이나 연애, 친밀한 관계를 의미합니다. 또한 사랑은 다른 사람에..."
3,friendship,👫,friendship은 친구 사이의 관계를 의미합니다. 이는 서로 신뢰하고 존중하는 ...
4,excited,🎉,"excited는 흥분한, 들뜬 감정이나 기분을 나타냅니다. 이 단어는 기대했던 일이..."
5,success,🎉,success는 성공을 의미합니다. 어떤 목표를 달성하거나 원하는 결과를 얻었을 때...
6,success,NaN,success는 성공이나 성취를 의미합니다. 어떤 목표를 달성하거나 원하는 결과를 ...
7,happy,😃,happy는 기쁨이나 만족감을 의미합니다. 그것은 긍정적인 감정이나 좋은 상태를 표...
8,success,🎉,success은 성공이나 성취를 의미합니다. 어떤 목표나 업적을 달성하거나 원하는 ...
9,laughter,😂,"laughter는 웃음을 의미합니다. 그것은 기쁨, 유쾌함, 웃기거나 재미있는 것으..."


In [34]:
fewshot_chain = {"input": RunnablePassthrough(),
                 "format_instruction": lambda _:format_instruction,
                 "example_1_json": lambda _:example_1_json, 
                 "example_2_json": lambda _:example_2_json}  | chat_prompt_template | model | StrOutputParser() # json그대로 저장 할것이기 때문에 

In [35]:
df["usage"] = df.apply(lambda row: fewshot_chain.invoke(row["word"]), axis=1)

In [36]:
df.to_excel("./words.xlsx", index=False)

In [37]:
df

,word,imoj,meaning,usage
0,effort,💪,effort은 노력이나 수고를 의미합니다. 그것은 어떤 일을 이루기 위해 투자한 에...,"{""conversation"": [{""role"": ""Person A"", ""conten..."
1,excitement,🎉,excitement은 흥분이나 설렘을 의미합니다. 어떤 일이나 사건이 특히 기쁘거나...,"{""conversation"": [{""role"": ""Person A"", ""conten..."
2,love,❤️,"love은 강한 애정이나 연애, 친밀한 관계를 의미합니다. 또한 사랑은 다른 사람에...","{""conversation"": [{""role"": ""Person A"", ""conten..."
3,friendship,👫,friendship은 친구 사이의 관계를 의미합니다. 이는 서로 신뢰하고 존중하는 ...,"{""conversation"": [{""role"": ""Person A"", ""conten..."
4,excited,🎉,"excited는 흥분한, 들뜬 감정이나 기분을 나타냅니다. 이 단어는 기대했던 일이...","{""conversation"": [{""role"": ""Person A"", ""conten..."
5,success,🎉,success는 성공을 의미합니다. 어떤 목표를 달성하거나 원하는 결과를 얻었을 때...,"{""conversation"": [{""role"": ""Person A"", ""conten..."
6,success,NaN,success는 성공이나 성취를 의미합니다. 어떤 목표를 달성하거나 원하는 결과를 ...,"{""conversation"": [{""role"": ""Person A"", ""conten..."
7,happy,😃,happy는 기쁨이나 만족감을 의미합니다. 그것은 긍정적인 감정이나 좋은 상태를 표...,"{""conversation"": [{""role"": ""Person A"", ""conten..."
8,success,🎉,success은 성공이나 성취를 의미합니다. 어떤 목표나 업적을 달성하거나 원하는 ...,"{""conversation"": [{""role"": ""Person A"", ""conten..."
9,laughter,😂,"laughter는 웃음을 의미합니다. 그것은 기쁨, 유쾌함, 웃기거나 재미있는 것으...","{""conversation"": [{""role"": ""Person A"", ""conten..."
